# Аттестация

## Загрузка датасета

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("atanaskanev/sqlite-sakila-sample-database")

print("Path to dataset files:", path)

/home/maksim/develops/python/MAKSIM/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 4.29M/4.29M [00:01<00:00, 2.29MB/s]

Extracting files...
Path to dataset files: /home/maksim/.cache/kagglehub/datasets/atanaskanev/sqlite-sakila-sample-database/versions/1


#### Импорты

In [3]:
import sqlite3
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [ ]:


# Подключение к SQLite
sqlite_db_path = "sakila.db"  # Путь к скачанной SQLite базе
sqlite_conn = sqlite3.connect(sqlite_db_path)

# Подключение к PostgreSQL
pg_user = "postgres"
pg_password = "postgres"
pg_host = "localhost"
pg_port = "5432"
pg_dbname = "sakila"

pg_engine = create_engine(f"postgresql://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_dbname}")

# Получение списка таблиц из SQLite
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, sqlite_conn)

# Перенос данных из каждой таблицы
for table_name in tables["name"]:
    df = pd.read_sql(f"SELECT * FROM {table_name}", sqlite_conn)
    df.to_sql(table_name, pg_engine, if_exists="replace", index=False)
    print(f"Таблица {table_name} успешно перенесена.")

# Закрываем соединения
sqlite_conn.close()
print("Перенос завершён!")
